<a href="https://colab.research.google.com/github/sujitojha1/ERA1/blob/main/S6/ERA1_Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [1]:
# Importing necessary libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    """Deep Convolutional Neural Network Model.
    
    This class builds a deep convolutional neural network model for image 
    classification using PyTorch. It contains three convolutional blocks 
    followed by a global average pooling layer and a fully connected layer.
    
    Attributes:
        conv1: A nn.Sequential object for the first convolutional block.
        conv2: A nn.Sequential object for the second convolutional block.
        conv3: A nn.Sequential object for the third convolutional block.
        gap: A nn.Sequential object for the global average pooling layer.
        fc: A nn.Sequential object for the fully connected layer.
    """

    def __init__(self):
        """Initializes the Net with its layers."""
        super(Net, self).__init__()

        # First Convolutional block
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),  # Convolution layer
            nn.ReLU(),  # Activation function
            nn.BatchNorm2d(16),  # Batch normalization
            nn.Conv2d(16, 16, 3, padding=1),  # Second convolution layer
            nn.ReLU(),  # Activation function
            nn.BatchNorm2d(16),  # Batch normalization
            nn.MaxPool2d(2, 2),  # Max pooling layer
            nn.Dropout(0.25)  # Dropout layer
        )
        
        # Second Convolutional block
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        # Third Convolutional block
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )

        # Global Average Pooling layer
        self.gap = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=10, kernel_size=(1, 1), padding=0),
        )
        
        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Linear(90, 10)
        )

    def forward(self, x):
        """Defines the computation performed at every call.
        
        Args:
            x: The input tensor with shape [batch_size, channels, height, width].
        
        Returns:
            A tensor of shape [batch_size, num_classes].
        """
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.gap(x)
        
        # Flattening the tensor
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 16, 28, 28]           2,320
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           2,320
             ReLU-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 16, 14, 14]           2,320
    

In [4]:
# Setting the seed for reproducibility
torch.manual_seed(1)

# Setting the batch size
batch_size = 128

# Checking if CUDA is available and setting PyTorch device to cuda or cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
# Loading the training data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# Loading the testing data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 372075454.13it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 67222915.55it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 75424420.58it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22625331.08it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    """Train the model.
    
    This function trains the model for one epoch.
    
    Args:
        model: The PyTorch model to train.
        device: The device (cpu or cuda) where the model will be trained.
        train_loader: The DataLoader for the training data.
        optimizer: The optimizer for the training.
        epoch: The current epoch number.
    """
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Epoch={epoch},loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    """Test the model.
    
    This function tests the model on test data and print test loss and accuracy.
    
    Args:
        model: The PyTorch model to test.
        device: The device (cpu or cuda) where the model will be tested.
        test_loader: The DataLoader for the test data.
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# Creating an instance of the Net class
model = Net().to(device)

# Using Stochastic Gradient Descent as the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# epochs
for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch=1,loss=0.048596736043691635 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.73it/s]



Test set: Average loss: 0.0449, Accuracy: 9846/10000 (98.4600%)



Epoch=2,loss=0.05985578894615173 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.78it/s]



Test set: Average loss: 0.0354, Accuracy: 9886/10000 (98.8600%)



Epoch=3,loss=0.13286419212818146 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 26.01it/s]


...


Test set: Average loss: 0.0189, Accuracy: 9937/10000 (99.3700%)



Epoch=19,loss=0.006624738220125437 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.76it/s]



Test set: Average loss: 0.0185, Accuracy: 9946/10000 (99.4600%)



Epoch=20,loss=0.07243505120277405 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.91it/s]



Test set: Average loss: 0.0179, Accuracy: 9944/10000 (99.4400%)

